In [116]:
from numpy import vstack
from pandas import read_csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, average_precision_score
from sklearn.metrics import confusion_matrix, recall_score, f1_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.optim import lr_scheduler
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import time
import copy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [117]:

thyroid = pd.read_csv('https://raw.githubusercontent.com/StatsGary/Data/main/thyroid_raw.csv')
#print(thyroid)
Y = thyroid.iloc[:, -1].values
X = thyroid.iloc[:,:-1]
print(len(X.columns), len(X))

# Made up label
labels = []
for idx, i in enumerate(Y):
    if idx % 4 != 0:
        labels.append('negative')
    else:
        labels.append('sick')


Y = labels
thyroid.iloc[:,-1] = pd.Series(Y)

thyroid.to_csv('thyroid_raw.csv', header=False, index=False)


26 2749


In [118]:

# Replicate data loader process
x = thyroid.values[:, :-1]
y = thyroid.values[:,-1]
x = x.astype('float32')
y_encoded = LabelEncoder().fit_transform(y)
y_encoded = y_encoded.astype('float32')
y_arrayed = y_encoded.reshape((len(y_encoded), 1))
print(y_encoded)


[1. 0. 0. ... 0. 0. 1.]


In [119]:
# Create a custom CSVDataset loader

class ThyroidCSVDataset(Dataset):
    #Constructor for initially loading
    def __init__(self,path):
        df = read_csv(path, header=None)
        # Store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1] #Assuming your outcome variable is in the first column
        self.X = self.X.astype('float32')
        # Label encode the target as values 1 and 0 or sick and not sick
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

    # Get the number of rows in the dataset
    def __len__(self):
        return len(self.X)
    # Get a row at an index
    def __getitem__(self,idx):
        return [self.X[idx], self.y[idx]]

    # Create custom class method - instead of dunder methods
    def split_data(self, split_ratio=0.2):
        test_size = round(split_ratio * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])


In [120]:
# Create model
class ThyroidMLP(Module):
    def __init__(self, n_inputs):
        super(ThyroidMLP, self).__init__()
        # First hidden layer
        self.hidden1 = Linear(n_inputs, 20)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # Second hidden layer
        self.hidden2 = Linear(20, 10)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # Third hidden layer
        self.hidden3 = Linear(10,1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()

    def forward(self, X):
        #Input to the first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # Second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # Third hidden layer
        X = self.hidden3(X)
        X = self.act3(X)
        return X


In [121]:
def prepare_thyroid_dataset(path):
    dataset = ThyroidCSVDataset(path)
    train, test = dataset.split_data(split_ratio=0.3)
    # Prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl



In [122]:
# Create training loop based off our custom class
def train_model(train_dl, model, epochs=100, lr=0.01, momentum=0.9, save_path='thyroid_best_model.pth'):
    # Define your optimisation function for reducing loss when weights are calculated 
    # and propogated through the network
    start = time.time()
    criterion = BCELoss()
    optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
    loss = 0.0

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch+1, epochs))
        print('-' * 10)
        model.train()
        # Iterate through training data loader
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data,1) #Get the class labels
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        torch.save(model, save_path)
    time_delta = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_delta // 60, time_delta % 60
    ))
    
    return model



In [123]:
import math
def evaluate_model(test_dl, model, beta=1.0):
    preds = []
    actuals = []

    for (i, (inputs, targets)) in enumerate(test_dl):
        #Evaluate the model on the test set
        yhat = model(inputs)
        #Retrieve a numpy weights array
        yhat = yhat.detach().numpy()
        # Extract the weights using detach to get the numerical values in an ndarray, instead of tensor
        #https://www.tutorialspoint.com/how-to-convert-a-pytorch-tensor-with-gradient-to-a-numpy-array
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # Round to get the class value i.e. sick vs not sick
        yhat = yhat.round()
        # Store the predictions in the empty lists initialised at the start of the class
        preds.append(yhat)
        actuals.append(actual)
    
    # Stack the predictions and actual arrays vertically
    preds, actuals = vstack(preds), vstack(actuals)
    #Calculate metrics
    cm = confusion_matrix(actuals, preds)
    # Get descriptions of tp, tn, fp, fn
    tn, fp, fn, tp = cm.ravel()
    total = sum(cm.ravel())
    
    metrics = {
        'accuracy': accuracy_score(actuals, preds),
        'AU_ROC': roc_auc_score(actuals, preds),
        'f1_score': f1_score(actuals, preds),
        'average_precision_score': average_precision_score(actuals, preds),
        'f_beta': ((1+beta**2) * precision_score(actuals, preds) * recall_score(actuals, preds)) / (beta**2 * precision_score(actuals, preds) + recall_score(actuals, preds)),
        'matthews_correlation_coefficient': (tp*tn - fp*fn) / math.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)),
        'precision': precision_score(actuals, preds),
        'recall': recall_score(actuals, preds),
        'true_positive_rate_TPR':recall_score(actuals, preds),
        'false_positive_rate_FPR':fp / (fp + tn) ,
        'false_discovery_rate': fp / (fp +tp),
        'false_negative_rate': fn / (fn + tp) ,
        'negative_predictive_value': tn / (tn+fn),
        'misclassification_error_rate': (fp+fn)/total ,
        'sensitivity': tp / (tp + fn),
        'specificity': tn / (tn + fp),
        #'confusion_matrix': confusion_matrix(actuals, preds), 
        'TP': tp,
        'FP': fp, 
        'FN': fn, 
        'TN': tn
    }
    return metrics, preds, actuals
        

In [124]:
# Create prediction routine
def predict(row, model):
    row = Tensor([row])
    yhat = model(row)
    # Get numpy array
    yhat = yhat.detach().numpy()
    return yhat

# Using the model

In [125]:

train_dl, test_dl = prepare_thyroid_dataset('https://raw.githubusercontent.com/StatsGary/Data/main/thyroid_raw.csv')

In [126]:
print(len(train_dl.dataset), len(test_dl.dataset))
print(train_dl.dataset)

1925 825


1925

In [127]:
# Specify the number of input dimensions
model = ThyroidMLP(26)
# Train the model
train_model(train_dl, model, 
            save_path='data/thyroid_model.pth', 
            epochs=150, 
            lr=0.01)



Epoch 1/150
----------
Epoch 2/150
----------
Epoch 3/150
----------
Epoch 4/150
----------
Epoch 5/150
----------
Epoch 6/150
----------
Epoch 7/150
----------
Epoch 8/150
----------
Epoch 9/150
----------
Epoch 10/150
----------
Epoch 11/150
----------
Epoch 12/150
----------
Epoch 13/150
----------
Epoch 14/150
----------
Epoch 15/150
----------
Epoch 16/150
----------
Epoch 17/150
----------
Epoch 18/150
----------
Epoch 19/150
----------
Epoch 20/150
----------
Epoch 21/150
----------
Epoch 22/150
----------
Epoch 23/150
----------
Epoch 24/150
----------
Epoch 25/150
----------
Epoch 26/150
----------
Epoch 27/150
----------
Epoch 28/150
----------
Epoch 29/150
----------
Epoch 30/150
----------
Epoch 31/150
----------
Epoch 32/150
----------
Epoch 33/150
----------
Epoch 34/150
----------
Epoch 35/150
----------
Epoch 36/150
----------
Epoch 37/150
----------
Epoch 38/150
----------
Epoch 39/150
----------
Epoch 40/150
----------
Epoch 41/150
----------
Epoch 42/150
----------
E

ThyroidMLP(
  (hidden1): Linear(in_features=26, out_features=20, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=20, out_features=10, bias=True)
  (act2): ReLU()
  (hidden3): Linear(in_features=10, out_features=1, bias=True)
  (act3): Sigmoid()
)

In [128]:
# Evaluate the model
results = evaluate_model(test_dl, model, beta=1)
model_metrics = results[0]
metrics_df = pd.DataFrame.from_dict(model_metrics, orient='index', columns=['metric'])
metrics_df.index.name = 'metric_type'
metrics_df.reset_index(inplace=True)
metrics_df.to_csv('confusion_matrix_thyroid.csv', index=False)
print(metrics_df)


                         metric_type      metric
0                           accuracy    0.723636
1                             AU_ROC    0.508414
2                           f1_score    0.142857
3            average_precision_score    0.242315
4                             f_beta    0.142857
5   matthews_correlation_coefficient    0.025917
6                          precision    0.275362
7                             recall    0.096447
8             true_positive_rate_TPR    0.096447
9            false_positive_rate_FPR    0.079618
10              false_discovery_rate    0.724638
11               false_negative_rate    0.903553
12         negative_predictive_value    0.764550
13      misclassification_error_rate    0.276364
14                       sensitivity    0.096447
15                       specificity    0.920382
16                                TP   19.000000
17                                FP   50.000000
18                                FN  178.000000
19                  

In [130]:
#                          metric_type      metric
# 0                           accuracy    0.727273
# 1                             AU_ROC    0.497436
# 2                           f1_score    0.096386
# 3            average_precision_score    0.235493
# 4                             f_beta    0.096386
# 5   matthews_correlation_coefficient   -0.008809
# 6                          precision    0.222222
# 7                             recall    0.061538
# 8             true_positive_rate_TPR    0.061538
# 9            false_positive_rate_FPR    0.066667
# 10              false_discovery_rate    0.777778
# 11               false_negative_rate    0.938462
# 12         negative_predictive_value    0.762646
# 13      misclassification_error_rate    0.272727
# 14                       sensitivity    0.061538
# 15                       specificity    0.933333
# 16                                TP   12.000000
# 17                                FP   42.000000
# 18                                FN  183.000000
# 19                                TN  588.000000

In [131]:
row = [0.8408678952719717,0.7480132415430958,-0.3366221139379705,-0.0938130059640389,-0.1101874782051067,-0.2098160394213988,-0.1260114177378201,-0.1118651062104989,-0.1274917875477927,-0.240146053214037,-0.2574472174396955,-0.0715198539852151,-0.0855764265990022,-0.1493202733578882,-0.0190692517849118,-0.2590488060984638,0.0,-0.1753175780014474,0.0,-0.9782211033008232,0.0,-1.3237957945784953,0.0,-0.6384998731458282,0.0,-1.209042232192488]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))


Predicted: 0.448 (class=0)


In [134]:
# Get the ionsphere data
train_dl, test_dl = prepare_thyroid_dataset('https://raw.githubusercontent.com/StatsGary/Data/main/ion.csv')
# Train the model
# Specify the number of input dimensions
model = ThyroidMLP(34)
# Train the model
train_model(train_dl, model, 
            save_path='data/ionsphere_model.pth', 
            epochs=150, 
            lr=0.01)


Epoch 1/150
----------
Epoch 2/150
----------
Epoch 3/150
----------
Epoch 4/150
----------
Epoch 5/150
----------
Epoch 6/150
----------
Epoch 7/150
----------
Epoch 8/150
----------
Epoch 9/150
----------
Epoch 10/150
----------
Epoch 11/150
----------
Epoch 12/150
----------
Epoch 13/150
----------
Epoch 14/150
----------
Epoch 15/150
----------
Epoch 16/150
----------
Epoch 17/150
----------
Epoch 18/150
----------
Epoch 19/150
----------
Epoch 20/150
----------
Epoch 21/150
----------
Epoch 22/150
----------
Epoch 23/150
----------
Epoch 24/150
----------
Epoch 25/150
----------
Epoch 26/150
----------
Epoch 27/150
----------
Epoch 28/150
----------
Epoch 29/150
----------
Epoch 30/150
----------
Epoch 31/150
----------
Epoch 32/150
----------
Epoch 33/150
----------
Epoch 34/150
----------
Epoch 35/150
----------
Epoch 36/150
----------
Epoch 37/150
----------
Epoch 38/150
----------
Epoch 39/150
----------
Epoch 40/150
----------
Epoch 41/150
----------
Epoch 42/150
----------
E

In [135]:
# Evaluate the model
def eval_model(test_dl, model, cm_out_name='confusion_mat.csv',
               beta=1, export_index=False):
    results = evaluate_model(test_dl, model, beta)
    model_metrics = results[0]
    metrics_df = pd.DataFrame.from_dict(model_metrics, orient='index', columns=['metric'])
    metrics_df.index.name = 'metric_type'
    metrics_df.reset_index(inplace=True)
    metrics_df.to_csv(cm_out_name, index=export_index)
    print(metrics_df)
    return metrics_df, model_metrics, results

results = eval_model(test_dl, model)
print(results[0])

                         metric_type     metric
0                           accuracy   0.923810
1                             AU_ROC   0.882353
2                           f1_score   0.946667
3            average_precision_score   0.898734
4                             f_beta   0.946667
5   matthews_correlation_coefficient   0.829016
6                          precision   0.898734
7                             recall   1.000000
8             true_positive_rate_TPR   1.000000
9            false_positive_rate_FPR   0.235294
10              false_discovery_rate   0.101266
11               false_negative_rate   0.000000
12         negative_predictive_value   1.000000
13      misclassification_error_rate   0.076190
14                       sensitivity   1.000000
15                       specificity   0.764706
16                                TP  71.000000
17                                FP   8.000000
18                                FN   0.000000
19                                TN  26

In [ ]:

#                          metric_type     metric
# 0                           accuracy   0.923810
# 1                             AU_ROC   0.882353
# 2                           f1_score   0.946667
# 3            average_precision_score   0.898734
# 4                             f_beta   0.946667
# 5   matthews_correlation_coefficient   0.829016
# 6                          precision   0.898734
# 7                             recall   1.000000
# 8             true_positive_rate_TPR   1.000000
# 9            false_positive_rate_FPR   0.235294
# 10              false_discovery_rate   0.101266
# 11               false_negative_rate   0.000000
# 12         negative_predictive_value   1.000000
# 13      misclassification_error_rate   0.076190
# 14                       sensitivity   1.000000
# 15                       specificity   0.764706
# 16                                TP  71.000000
# 17                                FP   8.000000
# 18                                FN   0.000000
# 19                                TN  26.000000

In [137]:
# Make prediction against model
row = [1,0,1,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

Predicted: 0.992 (class=1)
